In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

tampa_df = pd.read_csv ('tampa.csv')
read_dis_df = pd.read_csv ('disasters.csv')

In [7]:
# filter to match what is in the city
counties = ['Hillsborough County', 'Pinellas County']
filtered_df = read_dis_df[read_dis_df['County'].isin(counties)]

# get all unique hurricane names
hurricane_names = filtered_df['Declaration Title'].str.extract(r'(HURRICANE\s+\w+)', expand=False).dropna()
hurricane_names = hurricane_names.unique()

disaster_df = pd.DataFrame({'hurricanes': hurricane_names})

# Create a dictionary mapping Hurricane names to Declaration Dates
hurricane_date_mapping = {}

for hurricane_name in disaster_df['hurricanes']:
    # Find matching Declaration Titles (considering variations)
    matching_titles = filtered_df[filtered_df['Declaration Title'].str.contains(hurricane_name, case=False)]['Declaration Title']
    if not matching_titles.empty:
        # Use the first matching Declaration Date
        hurricane_date_mapping[hurricane_name] = filtered_df.loc[filtered_df['Declaration Title'] == matching_titles.iloc[0], 'Declaration Date'].values[0]

# add in declaration date to new df
disaster_df['date'] = pd.to_datetime(disaster_df['hurricanes'].map(hurricane_date_mapping))

# add 6 months prior and after
disaster_df['-6 months'] = disaster_df['date'] - pd.DateOffset(months=6)
disaster_df['+6 months'] = disaster_df['date'] + pd.DateOffset(months=6)

# convert tampa data to date


# check if in between 2 dates
#for hurricane_name in disaster_df['hurricanes']:

print(disaster_df)

           hurricanes       date  -6 months  +6 months
0    HURRICANE IDALIA 2023-08-31 2023-02-28 2024-02-29
1    HURRICANE NICOLE 2022-12-13 2022-06-13 2023-06-13
2       HURRICANE IAN 2022-09-29 2022-03-29 2023-03-29
3       HURRICANE ETA 2020-11-11 2020-05-11 2021-05-11
4    HURRICANE DORIAN 2019-08-30 2019-02-28 2020-02-29
5   HURRICANE MICHAEL 2018-10-09 2018-04-09 2019-04-09
6      HURRICANE IRMA 2017-09-10 2017-03-10 2018-03-10
7   HURRICANE HERMINE 2016-09-28 2016-03-28 2017-03-28
8   HURRICANE KATRINA 2005-09-05 2005-03-05 2006-03-05
9    HURRICANE JEANNE 2004-09-26 2004-03-26 2005-03-26
10  HURRICANE FRANCES 2004-09-04 2004-03-04 2005-03-04
11  HURRICANE CHARLEY 2004-08-13 2004-02-13 2005-02-13
12  HURRICANE GRORGES 1998-09-25 1998-03-25 1999-03-25
13    HURRICANE ELENA 1985-09-12 1985-03-12 1986-03-12
14   HURRICANE GLADYS 1968-11-07 1968-05-07 1969-05-07
15    HURRICANE SANDY 2012-10-30 2012-04-30 2013-04-30
16    HURRICANE IRENE 2011-08-27 2011-02-27 2012-02-27
17      HU